In [1]:
from array import array
from collections import deque

Будем хранить матрицу весов как список пар.

Матрица весов: 

$$
W = \begin{pmatrix}
     & A & B & C & \_ \\
   A & 5 & -2 & -1 & -5 \\
   B & -2 & 7 & -1 & -5 \\
   C & -1 & -1 & 7 & -5 \\
  \_ & -5 & -5 & -5 & -5 \\
\end{pmatrix}
$$

In [2]:
gap = '_'
gap_penalty = -5
weight_matrix = {
    ('A', gap):  gap_penalty,
    ('A', 'A'):  5,
    ('A', 'B'): -2,
    ('A', 'C'): -1,
    ('B', gap):  gap_penalty,
    ('B', 'A'): -2,
    ('B', 'B'):  7,
    ('B', 'C'): -1,
    ('C', gap):  gap_penalty,
    ('C', 'A'): -1,
    ('C', 'B'): -1,
    ('C', 'C'):  7,
    (gap, gap):  gap_penalty,
    (gap, 'A'):  gap_penalty,
    (gap, 'B'):  gap_penalty,
    (gap, 'C'):  gap_penalty,
}

In [8]:
# вспомогательная функция для преобразования двумерного индекса к одномурному.
def index2d_to_1d(i, j, n):
    return n * i + j

In [4]:
def alignment_weights(s, t, weight_matrix, gap='_'):
    m = len(s)
    n = len(t)
    
    C = array('h', [0]) * ((m + 1) * (n + 1))

    for j in range(n + 1):
        C[index2d_to_1d(0, j, n + 1)] += j * weight_matrix[(gap, gap)]
        
    for i in range(m + 1):
        C[index2d_to_1d(i, 0, n + 1)] += i * weight_matrix[(gap, gap)]
        
    i = 1
    while i <= m:
        j = 1
        while j <= n:
            C[index2d_to_1d(i, j, n + 1)] = max(
                C[index2d_to_1d(i - 1, j - 1, n + 1)] + weight_matrix[(s[i - 1], t[j - 1])],
                C[(n + 1) * i + (j - 1)] + weight_matrix[(gap, t[j - 1])],
                C[(n + 1) * (i - 1) + j] + weight_matrix[(s[i - 1], gap)]
            )
            j += 1
        i += 1
    return C

def alignment(s, t, weight_matrix, gap='_'):
    m = len(s)
    n = len(t)
    
    C = alignment_weights(s, t, weight_matrix, gap)
    
    s_a, t_a = deque(), deque()
    i, j = m, n
    while i > 0 and j > 0:
        if (C[index2d_to_1d(i - 1, j - 1, n + 1)] >= C[index2d_to_1d(i, j - 1, n + 1)] and
            C[index2d_to_1d(i - 1, j - 1, n + 1)] >= C[index2d_to_1d(i - 1, j, n + 1)]):
            s_a.appendleft(s[i - 1])
            t_a.appendleft(t[j - 1])
            i -= 1
            j -= 1
        elif C[index2d_to_1d(i, j - 1, n + 1)] > C[index2d_to_1d(i - 1, j, n + 1)]:
            s_a.appendleft(gap)
            t_a.appendleft(t[j - 1])
            j -= 1
        else:
            s_a.appendleft(s[i - 1])
            t_a.appendleft(gap)
            i -= 1
    
    return ''.join(s_a), ''.join(t_a)

In [5]:
alignment('ABC', 'AC', weight_matrix, gap=gap)

('ABC', 'AC_')

Увеличим штраф за мисметч B и С c -1 до -10.

$$
W' = \begin{pmatrix}
     & A & B & C & \_ \\
   A & 5 & -2 & -1 & -5 \\
   B & -2 & 7 & \Huge -10 & -5 \\
   C & -1 & -1 & 7 & -5 \\
  \_ & -5 & -5 & -5 & -5 \\
\end{pmatrix}
$$

In [6]:
weight_matrix[('B', 'C')] = -10

In [7]:
alignment('ABC', 'AC', weight_matrix, gap=gap)

('ABC', 'A_C')

Видно, что выравнивание изменилось. 